In [1]:
print("Welcome to my EMR Notebook!")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
41,application_1597998755054_0074,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Welcome to my EMR Notebook!

In [24]:
# Import libraries
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, \
                                  avg, from_unixtime, split, min, max, round, lit, mean
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

from pyspark.sql.types import FloatType
import datetime
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, hour, date_format, substring

import numpy as np
import time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Cancelled users

In [3]:
# THE BEST MODEL
metrics = spark.read.json('s3n://amosvoron-sparkify/gbt-cancelled-metrics-f22h.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+------------------+-----------------+------------------+
|                f1|         accuracy| weightedPrecision|   weightedRecall|               AUC|
+------------------+-----------------+------------------+-----------------+------------------+
|0.8994328175799097|0.902269152999326|0.8993354342260549|0.902269152999326|0.8332933475246603|
+------------------+-----------------+------------------+-----------------+------------------+

In [5]:
# [Logistic Regression]    
# 8 mini data set features
# hyperparams: elasticNetParam=0.0, maxIter=10, regParam=0.1
metrics_01 = spark.read.json('s3n://amosvoron-sparkify/lr-metrics.json')
metrics_01.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+------------------+------------------+------------------+
|               f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+-----------------+------------------+------------------+------------------+------------------+
|0.774112692852762|0.8126263760952595|0.8012311289019549|0.8126263760952596|0.6116784733854458|
+-----------------+------------------+------------------+------------------+------------------+

In [86]:
metrics_01 = metrics_01 \
    .withColumn('Nº', lit('1')) \
    .withColumn('ML method', lit('Logistic Regression')) \
    .withColumn('features', lit('8')) \
    .withColumn('note', lit('mini dataset features')) \
    .withColumn('elasticNetParam', lit('0.0')) \
    .withColumn('maxIter', lit('10')) \
    .withColumn('regParam', lit('0.1')) \
    .withColumn('maxDepth', lit('N/A')) \
    .withColumn('numTrees', lit('N/A'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# [Random Forest]    
# 29 features (from mini data set)
# hyperparams: default
#metrics_02 = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics.json')
#metrics_02.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()
values_02 = [0.688967, 0.853741, 0.830521, 0.859686, 0.853741] 
columns_02 = ['AUC', 'accuracy', 'f1', 'weightedPrecision', 'weightedRecall']
rdd = spark.sparkContext.parallelize([values_02])
metrics_02 = rdd.toDF(columns_02) 
metrics_02.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------+-----------------+--------------+
|     AUC|accuracy|      f1|weightedPrecision|weightedRecall|
+--------+--------+--------+-----------------+--------------+
|0.688967|0.853741|0.830521|         0.859686|      0.853741|
+--------+--------+--------+-----------------+--------------+

In [87]:
metrics_02 = metrics_02 \
    .withColumn('Nº', lit('2')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('29')) \
    .withColumn('note', lit('mini dataset features')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('default')) \
    .withColumn('numTrees', lit('default'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
# [Random Forest]    
# 3 features (chosen by analysis: AvgSessionGap, SessionCount, LogoutPerHour)
# hyperparams: maxDepth=6, numTrees=20
metrics_03 = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics-test.json')
metrics_03.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.8697473174840196|0.8782296113232981|0.8742039209639965|0.8782296113232981|0.7698629141075749|
+------------------+------------------+------------------+------------------+------------------+

In [100]:
metrics_03 = metrics_03 \
    .withColumn('Nº', lit('3')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('3')) \
    .withColumn('note', lit('*')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('6')) \
    .withColumn('numTrees', lit('20'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# [Random Forest]    
# 22 features
# hyperparams: maxDepth=6, numTrees=20
metrics_04 = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics-f22.json')
metrics_04.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+-----------------+------------------+------------------+
|                f1|          accuracy|weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+-----------------+------------------+------------------+
|0.8386651060216977|0.8577847674679847|0.858875141192138|0.8577847674679848|0.7055267657733111|
+------------------+------------------+-----------------+------------------+------------------+

In [89]:
metrics_04 = metrics_04 \
    .withColumn('Nº', lit('4')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('22')) \
    .withColumn('note', lit('big dataset features')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('6')) \
    .withColumn('numTrees', lit('20'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# [Random Forest]    
# 14 features (reduction from f22)
# hyperparams: maxDepth=6, numTrees=20
metrics_05 = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics-f14.json')
metrics_05.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+------------------+------------------+------------------+
|               f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+-----------------+------------------+------------------+------------------+------------------+
|0.858039909315744|0.8696922039991013|0.8661328993108217|0.8696922039991013|0.7457577238042062|
+-----------------+------------------+------------------+------------------+------------------+

In [90]:
metrics_05 = metrics_05 \
    .withColumn('Nº', lit('5')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('14')) \
    .withColumn('note', lit('features derived from Nº4')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('6')) \
    .withColumn('numTrees', lit('20'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# [Random Forest]    
# 9 features (reduction from f14)
# hyperparams: maxDepth=6, numTrees=20
metrics_06 = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics-f9.json')
metrics_06.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------------+-----------------+----------------+------------------+
|                f1|        accuracy|weightedPrecision|  weightedRecall|               AUC|
+------------------+----------------+-----------------+----------------+------------------+
|0.8608286816841433|0.87059087845428|0.865639943751543|0.87059087845428|0.7552823434301136|
+------------------+----------------+-----------------+----------------+------------------+

In [91]:
metrics_06 = metrics_06 \
    .withColumn('Nº', lit('6')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('9')) \
    .withColumn('note', lit('features derived from Nº5')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('6')) \
    .withColumn('numTrees', lit('20'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# [Random Forest]    
# 9 features (reduction from f14)
# hyperparams: maxDepth=6, numTrees=50
metrics_07 = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics-f9h.json')
metrics_07.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+-----------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|              AUC|
+------------------+------------------+------------------+------------------+-----------------+
|0.8682717979279954|0.8777802740957088|0.8745521507864166|0.8777802740957088|0.764206012702401|
+------------------+------------------+------------------+------------------+-----------------+

In [92]:
metrics_07 = metrics_07 \
    .withColumn('Nº', lit('7')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('9')) \
    .withColumn('note', lit('features derived from Nº5')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('6')) \
    .withColumn('numTrees', lit('50'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# [Random Forest]    
# 6 features (reduction from f9)
# hyperparams: maxDepth=6, numTrees=50
metrics_08 = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics-f6h.json')
metrics_08.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+------------------+------------------+------------------+
|                f1|         accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+-----------------+------------------+------------------+------------------+
|0.8794465581711159|0.886093012806111|0.8824006435699202|0.8860930128061111|0.7885243637167317|
+------------------+-----------------+------------------+------------------+------------------+

In [93]:
metrics_08 = metrics_08 \
    .withColumn('Nº', lit('8')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('6')) \
    .withColumn('note', lit('features derived from Nº6/Nº7')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('6')) \
    .withColumn('numTrees', lit('50'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# [GBT]    
# 22 features
# hyperparams: maxIter=50, maxDepth=5
metrics_09 = spark.read.json('s3n://amosvoron-sparkify/gbt-cancelled-metrics-f22.json')
metrics_09.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+------------------+------------------+------------------+
|               f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+-----------------+------------------+------------------+------------------+------------------+
|0.893976019228649|0.8973264434958437|0.8939976348676694|0.8973264434958437|0.8233115810534152|
+-----------------+------------------+------------------+------------------+------------------+

In [94]:
metrics_09 = metrics_09 \
    .withColumn('Nº', lit('9')) \
    .withColumn('ML method', lit('GBTClassifier')) \
    .withColumn('features', lit('22')) \
    .withColumn('note', lit('Nº4 features')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('50')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('5')) \
    .withColumn('numTrees', lit('N/A'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# [GBT]    
# 22 features
# hyperparams: maxIter=200, maxDepth=5
metrics_10 = spark.read.json('s3n://amosvoron-sparkify/gbt-cancelled-metrics-f22h.json')
metrics_10.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+------------------+-----------------+------------------+
|                f1|         accuracy| weightedPrecision|   weightedRecall|               AUC|
+------------------+-----------------+------------------+-----------------+------------------+
|0.8994328175799097|0.902269152999326|0.8993354342260549|0.902269152999326|0.8332933475246603|
+------------------+-----------------+------------------+-----------------+------------------+

In [95]:
metrics_10 = metrics_10 \
    .withColumn('Nº', lit('10')) \
    .withColumn('ML method', lit('GBTClassifier')) \
    .withColumn('features', lit('22')) \
    .withColumn('note', lit('Nº4 features')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('200')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('5')) \
    .withColumn('numTrees', lit('N/A'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
# [GBT]    
# 16 features (reduction from f22)
# hyperparams: maxIter=150, maxDepth=5
metrics_11 = spark.read.json('s3n://amosvoron-sparkify/gbt-cancelled-metrics-f16h.json')
metrics_11.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.8993786653444871|0.9024938216131206|0.8995485523750624|0.9024938216131206|0.8312910036292573|
+------------------+------------------+------------------+------------------+------------------+

In [96]:
metrics_11 = metrics_11 \
    .withColumn('Nº', lit('11')) \
    .withColumn('ML method', lit('GBTClassifier')) \
    .withColumn('features', lit('16')) \
    .withColumn('note', lit('features derived from Nº10')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('150')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('5')) \
    .withColumn('numTrees', lit('N/A'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# [Logistic Regression]    
# 16 features (reduction from f22)
# hyperparams: elasticNetParam=0.0, maxIter=50, regParam=0.1
metrics_12 = spark.read.json('s3n://amosvoron-sparkify/lr-cancelled-metrics-f22h.json')
metrics_12.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.7505143102418945|0.7984722534261963|0.7780010603901284|0.7984722534261963|0.5803780185650474|
+------------------+------------------+------------------+------------------+------------------+

In [97]:
metrics_12 = metrics_12 \
    .withColumn('Nº', lit('12')) \
    .withColumn('ML method', lit('Logistic Regression')) \
    .withColumn('features', lit('16')) \
    .withColumn('note', lit('features derived from Nº10')) \
    .withColumn('elasticNetParam', lit('0.0')) \
    .withColumn('maxIter', lit('50')) \
    .withColumn('regParam', lit('0.1')) \
    .withColumn('maxDepth', lit('N/A')) \
    .withColumn('numTrees', lit('N/A'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# TODO

# [GBT]    
# 16 features (reduction from f22)
# hyperparams: maxIter=200, maxDepth=5
metrics_12 = spark.read.json('s3n://amosvoron-sparkify/gbt-cancelled-metrics-f16h2.json')
metrics_12.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

In [ ]:
# TODO

# [GBT]    
# 22 or 16 features (reduction from f22)
# hyperparams: maxIter=50, maxDepth=2
metrics_12 = spark.read.json('s3n://amosvoron-sparkify/gbt-cancelled-metrics-f16h3.json')
metrics_12.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

### Final Result

In [136]:
metrics_cancelled = metrics_01.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') \
    .union(metrics_02.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_03.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_04.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_05.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_06.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_07.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_08.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_09.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_10.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_11.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_12.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
metrics_cancelled \
    .select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', round('f1', 6).alias('F1'), 'note') \
    .show(20, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------------+--------+---------------+-------+--------+--------+--------+--------+-----------------------------+
|Nº |ML method          |features|elasticNetParam|maxIter|regParam|maxDepth|numTrees|F1      |note                         |
+---+-------------------+--------+---------------+-------+--------+--------+--------+--------+-----------------------------+
|1  |Logistic Regression|8       |0.0            |10     |0.1     |N/A     |N/A     |0.774113|mini dataset features        |
|2  |Random Forest      |29      |N/A            |N/A    |N/A     |default |default |0.830521|mini dataset features        |
|3  |Random Forest      |3       |N/A            |N/A    |N/A     |6       |20      |0.869747|*                            |
|4  |Random Forest      |22      |N/A            |N/A    |N/A     |6       |20      |0.838665|big dataset features         |
|5  |Random Forest      |14      |N/A            |N/A    |N/A     |6       |20      |0.85804 |features derived from Nº4    |


### Best model for cancelled users

In [156]:
metrics_10.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+------------------+-----------------+------------------+
|                f1|         accuracy| weightedPrecision|   weightedRecall|               AUC|
+------------------+-----------------+------------------+-----------------+------------------+
|0.8994328175799097|0.902269152999326|0.8993354342260549|0.902269152999326|0.8332933475246603|
+------------------+-----------------+------------------+-----------------+------------------+

In [161]:
importances_10 = spark.read.json('s3n://amosvoron-sparkify/gbt-cancelled-feature-importances-f22h.json')
importances_10.sort(desc('importance')).select('feature', round('importance', 6).alias('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+----------+
|feature                |importance|
+-----------------------+----------+
|AvgSessionGap          |0.114945  |
|NonSongCount           |0.064948  |
|SessionCount           |0.060762  |
|RollAdvertPerHour      |0.057997  |
|ThumbsUpCount          |0.057046  |
|ThumbsDownPerHour      |0.055785  |
|SettingsPerSessionHour |0.055747  |
|RollAdvertCount        |0.048809  |
|DowngradePerSessionHour|0.047659  |
|DowngradePerHour       |0.046806  |
|LogoutPerHour          |0.042215  |
|HomeCount              |0.040791  |
|HelpCount              |0.038668  |
|RollAdvertPerDay       |0.03714   |
|HomePerHour            |0.035674  |
|SettingsPerHour        |0.033925  |
|AboutPerSessionHour    |0.033717  |
|UpgradePerSessionHour  |0.032103  |
|AboutCount             |0.027766  |
|ErrorPerSessionHour    |0.027089  |
|SubmitUpgradePerHour   |0.021087  |
|SubmitDowngradePerHour |0.019319  |
+-----------------------+----------+

## Downgraded users

In [105]:
# [Random Forest]    
# 22 features (big data features)
# hyperparams: maxDepth=6, numTrees=100
metrics_15 = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-22h.json')
metrics_15.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+-----------------+------------------+
|                f1|          accuracy| weightedPrecision|   weightedRecall|               AUC|
+------------------+------------------+------------------+-----------------+------------------+
|0.8137938477161866|0.8303751965850371|0.8179372014878397|0.830375196585037|0.6879226254158419|
+------------------+------------------+------------------+-----------------+------------------+

In [147]:
metrics_15 = metrics_15 \
    .withColumn('Nº', lit('15')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('22')) \
    .withColumn('note', lit('big dataset features**')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('6')) \
    .withColumn('numTrees', lit('100'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
# [Random Forest]    
# 13 features (big data features)
# hyperparams: maxDepth=6, numTrees=100
metrics_16 = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-13h.json')
metrics_16.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.8243558706304979|0.8346439002471355|0.8235744592195376|0.8346439002471354|0.7150563615408072|
+------------------+------------------+------------------+------------------+------------------+

In [139]:
metrics_16 = metrics_16 \
    .withColumn('Nº', lit('16')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('13')) \
    .withColumn('note', lit('features derived from Nº15')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('6')) \
    .withColumn('numTrees', lit('50'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
# [Random Forest]    
# 9 features (big data features)
# hyperparams: maxDepth=6, numTrees=100
metrics_17 = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-9h.json')
metrics_17.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.8084889296285297|0.8236351381711975|0.8094240821667913|0.8236351381711975|0.6849522237670034|
+------------------+------------------+------------------+------------------+------------------+

In [140]:
metrics_17 = metrics_17 \
    .withColumn('Nº', lit('17')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('9')) \
    .withColumn('note', lit('features derived from Nº16')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('6')) \
    .withColumn('numTrees', lit('20'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
# [Random Forest]    
# 11 features (big data features)
# hyperparams: maxDepth=6, numTrees=100
metrics_18 = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-11h.json')
metrics_18.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+------------------+------------------+------------------+
|               f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+-----------------+------------------+------------------+------------------+------------------+
|0.824815310854039|0.8339698944057515|0.8233021851946406|0.8339698944057515|0.7187978616553737|
+-----------------+------------------+------------------+------------------+------------------+

In [141]:
metrics_18 = metrics_18 \
    .withColumn('Nº', lit('18')) \
    .withColumn('ML method', lit('Random Forest')) \
    .withColumn('features', lit('11')) \
    .withColumn('note', lit('features derived from Nº16')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('N/A')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('6')) \
    .withColumn('numTrees', lit('50'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
# [GBT]    
# 22 features (big data features)
# hyperparams: default
metrics_19 = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-22-gbt.json')
metrics_19.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+-----------------+------------------+------------------+
|                f1|          accuracy|weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+-----------------+------------------+------------------+
|0.8148460890342518|0.8292518535160638|0.816390590460442|0.8292518535160638|0.6938103089160288|
+------------------+------------------+-----------------+------------------+------------------+

In [148]:
metrics_19 = metrics_19 \
    .withColumn('Nº', lit('19')) \
    .withColumn('ML method', lit('GBTClassifier')) \
    .withColumn('features', lit('22')) \
    .withColumn('note', lit('big dataset features**')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('default')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('default')) \
    .withColumn('numTrees', lit('N/A'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
# [GBT]    
# 22 features (big data features)
# hyperparams: default
metrics_20 = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-22h-gbt.json')
metrics_20.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.8187400219006676|0.8281285104470906|0.8167142782328922|0.8281285104470905|0.7108387931871132|
+------------------+------------------+------------------+------------------+------------------+

In [149]:
metrics_20 = metrics_20 \
    .withColumn('Nº', lit('20')) \
    .withColumn('ML method', lit('GBTClassifier')) \
    .withColumn('features', lit('22')) \
    .withColumn('note', lit('big dataset features**')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('200')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('5')) \
    .withColumn('numTrees', lit('N/A'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
# [GBT]    
# 12 features (big data features)
# hyperparams: default
metrics_21 = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-metrics-12h-gbt.json')
metrics_21.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+------------------+------------------+
|                f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+------------------+------------------+------------------+
|0.8216342673718903|0.8290271849022691|0.8190860749874386|0.8290271849022692|0.7201242773554035|
+------------------+------------------+------------------+------------------+------------------+

In [144]:
metrics_21 = metrics_21 \
    .withColumn('Nº', lit('21')) \
    .withColumn('ML method', lit('GBTClassifier')) \
    .withColumn('features', lit('12')) \
    .withColumn('note', lit('features derived from Nº20')) \
    .withColumn('elasticNetParam', lit('N/A')) \
    .withColumn('maxIter', lit('50')) \
    .withColumn('regParam', lit('N/A')) \
    .withColumn('maxDepth', lit('5')) \
    .withColumn('numTrees', lit('N/A'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
metrics_downgraded = metrics_15.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') \
    .union(metrics_16.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_17.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_18.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_19.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_20.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') ) \
    .union(metrics_21.select('Nº', 'ML method', 'features', 'elasticNetParam', 'maxIter', 'regParam', \
            'maxDepth', 'numTrees', 'f1', 'note') )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
metrics_downgraded.show(20, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------+--------+---------------+-------+--------+--------+--------+------------------+--------------------------+
|Nº |ML method    |features|elasticNetParam|maxIter|regParam|maxDepth|numTrees|f1                |note                      |
+---+-------------+--------+---------------+-------+--------+--------+--------+------------------+--------------------------+
|15 |Random Forest|22      |N/A            |N/A    |N/A     |6       |100     |0.8137938477161866|big dataset features**    |
|16 |Random Forest|13      |N/A            |N/A    |N/A     |6       |50      |0.8243558706304979|features derived from Nº15|
|17 |Random Forest|9       |N/A            |N/A    |N/A     |6       |20      |0.8084889296285297|features derived from Nº16|
|18 |Random Forest|11      |N/A            |N/A    |N/A     |6       |50      |0.824815310854039 |features derived from Nº16|
|19 |GBTClassifier|22      |N/A            |default|N/A     |default |N/A     |0.8148460890342518|big dataset features

### Best model for downgraded users

In [152]:
metrics_18.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+------------------+------------------+------------------+
|               f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+-----------------+------------------+------------------+------------------+------------------+
|0.824815310854039|0.8339698944057515|0.8233021851946406|0.8339698944057515|0.7187978616553737|
+-----------------+------------------+------------------+------------------+------------------+

In [155]:
importances_18 = spark.read.json('s3n://amosvoron-sparkify/rfc-downgraded-feature-importances-11h.json')
importances_18.sort(desc('importance')).select('feature', round('importance', 6).alias('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+
|feature          |importance|
+-----------------+----------+
|NonSongCount     |0.264593  |
|RollAdvertCount  |0.237722  |
|RollAdvertPerHour|0.111845  |
|LogCount         |0.097003  |
|LogoutCount      |0.083322  |
|TotalSongLength  |0.051403  |
|SongCount        |0.049966  |
|ThumbsDownCount  |0.044441  |
|UniqueSongCount  |0.029354  |
|HomePerHour      |0.015263  |
|AddFriendCount   |0.015088  |
+-----------------+----------+